In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import dataset
df = pd.read_csv('C:/Users/hacke/Downloads/train.csv', sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          309 non-null    int64 
 1   sex         309 non-null    object
 2   age         309 non-null    int64 
 3   address     309 non-null    object
 4   famsize     309 non-null    object
 5   Pstatus     309 non-null    object
 6   Medu        309 non-null    int64 
 7   Fedu        309 non-null    int64 
 8   Mjob        309 non-null    object
 9   Fjob        309 non-null    object
 10  reason      309 non-null    object
 11  guardian    309 non-null    object
 12  traveltime  309 non-null    int64 
 13  studytime   309 non-null    int64 
 14  failures    309 non-null    int64 
 15  schoolsup   309 non-null    object
 16  famsup      309 non-null    object
 17  paid        309 non-null    object
 18  activities  309 non-null    object
 19  nursery     309 non-null    object
 20  higher    

In [ ]:
# Encoding categorical variables using LabelEncoder()
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
# Choose categorial columns
cols = ['sex', 'address', 'famsize', 'Pstatus','Mjob','Fjob','reason','guardian','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic']
# Encode labels of multiple columns at once
df[cols] = df[cols].apply(LabelEncoder().fit_transform)
df.head(5)


,id,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1
0,1,0,16,1,0,1,4,4,3,3,...,1,0,3,2,3,1,2,2,6,13
1,2,1,15,1,0,1,2,2,2,2,...,1,0,4,2,2,1,2,5,2,12
2,3,0,16,1,0,1,4,3,4,1,...,1,0,3,4,4,2,4,4,2,10
3,4,0,17,1,0,0,4,4,2,4,...,0,0,4,1,4,1,1,1,6,6
4,5,1,15,1,0,1,3,4,2,2,...,1,0,3,4,3,1,2,4,6,14


In [ ]:
# splitting the data into training and validation data (85% and 15%)
X_train, X_valid, y_train, y_valid = train_test_split(df.drop('G1', axis =1), df['G1'], test_size = 0.15, random_state=42)

In [ ]:
# Linear regression model
from sklearn import metrics
from sklearn.metrics import r2_score
model = LinearRegression()
model.fit(X_train,y_train)
pred = model.predict(X_valid)
mse_lr = mean_squared_error(y_valid,pred)
print(mse_lr)

6.825664972328297


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,\
RandomizedSearchCV

# RF model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# predict the model
y_pred = model.predict(X_valid)

# RF with different options for parameters
param_grid = {'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9]}

# Cross-validation
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_model_rf = grid_search.best_estimator_
best_params_rf = grid_search.best_params_
print(f"Best parameters for RF model:\n{best_model_rf}")

Best parameters for RF model:
RandomForestRegressor(max_depth=6, max_features=None, max_leaf_nodes=9,
                      n_estimators=50, random_state=42)


In [ ]:
# Fit validation dataset
y_pred_grid_rf = best_model_rf.predict(X_valid)
mse_rf= mean_squared_error(y_valid,y_pred_grid )
print(mse_rf)


6.187575150742541


In [ ]:
if mse_lr < mse_rf:
    best_model = model
    model_name = 'Linear Regression'
else:
    best_model = best_model_rf
    model_name = 'Random Forest'

print(f'The best model is {model_name} with MSE of {mse_lr if mse_lr < mse_rf else mse_rf}')

The best model is Random Forest with MSE of 6.187575150742541


In [ ]:
# Build a forecast data from the test file
# Load test.csv file
df_test = pd.read_csv('C:/Users/hacke/Downloads/test.csv', sep=';')

# Encoding categorical variables using LabelEncoder()
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
# Choose categorial columns
cols = ['sex', 'address', 'famsize', 'Pstatus','Mjob','Fjob','reason','guardian','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic']
# Encode labels of multiple columns at once
df_test[cols] = df_test[cols].apply(LabelEncoder().fit_transform)

#Predictions on test data using RF model
pred_test = best_model_rf.predict(df_test)

# Create DataFrame with student ID an grade predictions
predictions = pd.DataFrame({'id': df_test['id'], 'G1': pred_test})
print(predictions)

# Save to CSV file

predictions.to_csv('C:/Users/hacke/Downloads/export_dataframe.csv', index=False, sep=';')

     id         G1
0   310  10.298360
1   311  11.896745
2   312  11.334900
3   313  12.685672
4   314  10.831194
5   315  12.211917
6   316  11.918682
7   317  12.136896
8   318  10.971415
9   319  12.987964
10  320  11.426628
11  321  10.800201
12  322  11.949781
13  323  10.665677
14  324  12.090119
15  325   7.478609
16  326  12.578072
17  327  11.041923
18  328  12.886224
19  329  11.920432
20  330  10.842378
21  331  11.282137
22  332  10.985529
23  333  10.483399
24  334   9.595466
25  335  12.697674
26  336  11.931906
27  337  11.671689
28  338  10.730310
29  339  12.429120
30  340  11.714082
31  341  11.957243
32  342   8.873887
33  343   9.395241
34  344  11.956252
35  345   7.704572
36  346   9.769935
37  347  11.557240
38  348  11.913677
39  349  12.104726
